In [1]:
from datasets import load_dataset
dataset = load_dataset("nrishabh/geolife", cache_dir="/home/rnanawa1/GeoLifeLearn/data")

Generating train split:   0%|          | 0/65925 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11390 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21634 [00:00<?, ? examples/s]

In [13]:
testds = dataset['test']

In [14]:
import wandb
import torch
import argparse
import numpy as np
from datetime import datetime
from dotenv import load_dotenv
from datasets import load_dataset, Image
from sklearn.metrics import accuracy_score
from transformers import TrainingArguments, Trainer
from transformers import ViTImageProcessor, ViTForImageClassification
from torchvision.transforms import Normalize, Resize, ToTensor, Compose


In [27]:
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch32-224-in21k")

mu, sigma = processor.image_mean, processor.image_std  # get default mu,sigma
size = processor.size

norm = Normalize(mean=mu, std=sigma)  # normalize image pixels range to [-1,1]

# resize 3x32x32 to 3x224x224 -> convert to Pytorch tensor -> normalize
_transf = Compose([Resize(size["height"]), ToTensor(), norm])

# apply transforms to PIL Image and store it to 'pixels' key
def transf(arg):
    try:
        arg["pixels"] = [_transf(image.convert("RGB")) for image in arg["image"]]
        return arg
    except KeyError:
        return arg


testds.set_transform(transf)

/home/rnanawa1/.conda/envs/geolife/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
testds['label']

[23,
 19,
 8,
 23,
 2,
 0,
 23,
 9,
 7,
 22,
 21,
 1,
 15,
 17,
 17,
 7,
 11,
 2,
 6,
 5,
 10,
 8,
 1,
 19,
 21,
 15,
 24,
 3,
 12,
 24,
 6,
 5,
 18,
 5,
 2,
 14,
 1,
 2,
 4,
 23,
 20,
 23,
 2,
 21,
 11,
 11,
 11,
 22,
 2,
 21,
 23,
 11,
 19,
 10,
 12,
 19,
 15,
 10,
 3,
 10,
 12,
 20,
 16,
 7,
 11,
 9,
 21,
 18,
 19,
 0,
 10,
 8,
 22,
 2,
 16,
 15,
 22,
 5,
 22,
 17,
 11,
 17,
 12,
 7,
 2,
 0,
 8,
 19,
 5,
 22,
 24,
 23,
 14,
 21,
 15,
 7,
 12,
 5,
 4,
 18,
 16,
 3,
 4,
 19,
 2,
 16,
 16,
 21,
 2,
 20,
 13,
 11,
 18,
 24,
 13,
 23,
 24,
 22,
 2,
 14,
 8,
 21,
 20,
 7,
 19,
 23,
 19,
 18,
 23,
 15,
 23,
 8,
 16,
 14,
 2,
 11,
 1,
 2,
 18,
 20,
 15,
 19,
 2,
 5,
 14,
 16,
 13,
 21,
 21,
 1,
 9,
 14,
 21,
 6,
 8,
 5,
 2,
 22,
 1,
 17,
 2,
 7,
 5,
 7,
 20,
 9,
 3,
 19,
 7,
 19,
 21,
 20,
 1,
 7,
 18,
 11,
 2,
 2,
 12,
 16,
 20,
 10,
 23,
 20,
 4,
 3,
 6,
 8,
 9,
 4,
 8,
 12,
 6,
 12,
 9,
 18,
 14,
 19,
 6,
 24,
 14,
 14,
 3,
 19,
 7,
 24,
 12,
 13,
 1,
 13,
 2,
 5,
 9,
 24,
 20,
 1,
 19,
